# Real or Not? NLP with Disaster Tweets
Predict which Tweets are about real disasters and which ones are not

ในปัจจุบันนี้ไม่ว่าบนโลกจะเกิดเหตุการณ์อะไรขึ้น ผู้คนมักใช้ Tweeter ในการติดตามข่าวสารที่เกิดขึ้นเป็นจำนวนมากซึ่ง เป็นแหล่งข่าวที่มีการอัพเดตอยู่ตลอดและว่องไวและผู้คนมักจะ tweet ข้อความข่าวสารเหล่านั้น โดยเฉพาะข่าวสารที่เกี่ยวกับภัยพิบัติ tweeter กลายเป็นแอพพลิเคชั่นสำคัญที่เอาไว้ใช้เฝ้าระวังเหตุฉุกเฉินแบบเรียลไทม์ แต่คำบางคำที่อาจจะเกี่ยวข้องกับภัยพิบัตินั้น ก็นำมาใช้เขียนในเชิงเปรียบเทียบที่คนเราสามารถอ่านแล้วเข้าใจได้ แต่ machine นั้นก็ไม่สามารถเข้าใจได้อย่างชัดเจน และเมื่อมีปัญหาที่เกิดขึ้น ทางกลุ่มเราจึงได้เข้าร่วมการแข่งขันการสร้างโมเดลที่คาดการณ์ว่าทวีตใดเกี่ยวกับภัยพิบัติจริงและทวีตใดที่ไม่ใช่

# Data Set
ประกอบด้วย

1. sample_submission.csv ที่เก็บข้อมูลของ id และ target ที่บ่งบอกว่า id นั้นได้พูดถึงภัยพิบัติจริงหรือไม่ ให้ 1 พูดถึงภัยพิบัติจริงและ 0 ไม่ได้พูดถึงภัยพิบัติ

2. test.csv ชุดการทดสอบ  ซึ่งประกอบด้วย
>     1. id : การระบุเลขที่มีการโพสต์ที่ไม่ซ้ำกัน
> 
>     2. Keyword : คำสำคัญสำหรับการทำนาย
> 
>     3. location : ตำแหน่งที่อยู่ของผู้ใช้งานทวีต
> 
>     4. text : ข้อความที่ผู้ใช้โพสต์
> 
3. train.csv ชุดการฝึก  ซึ่งประกอบด้วย
>     1. id : การระบุเลขที่มีการโพสต์ที่ไม่ซ้ำกัน
> 
>     2. Keyword : คำสำคัญสำหรับการทำนาย
> 
>     3. location : ตำแหน่งที่อยู่ของผู้ใช้งานทวีต
> 
>     4. text : ข้อความที่ผู้ใช้โพสต์
> 
>     5. target : สิ่งนี้บ่งชี้ว่าทวีตนั้นเกี่ยวกับภัยพิบัติจริงให้เป็น (1) หรือไม่จริงให้เป็น (0)

# Data Processing
ขั้นตอนแรกเราได้ทำการ import module ต่างๆที่จำเป็นต้องใช้ในโปรแกรม 
*  numpy สำหรับการคำนวณทางคณิตศาสตร์ 
*  pandas ที่ทำให้เราจัดการกับข้อมูลต่างๆได้ง่ายขึ้นและสามารถแสดงข้อมูลออกมาเป็นตารางได้ 
* nltk เป็นเครื่องมือการประมวลผลภาษาธรรมชาติ เพื่อให้คอมพิวเตอร์สามารถตีความและเข้าใจภาษามนุษย์ได้ 
* re ที่ทำให้สามารถค้นหากลุ่มตัวหนังสือที่มีรูปแบบตามที่ต้องการจากข้อความหรือกลุ่มตัวอักษรได้ 
* string เป็นลำดับของตัวอักษรหลายตัวเรียงต่อกัน ซึ่งในภาษา Python นั้นการที่จะประกาศ String
  ค่าของมันจะอยู่ในเครื่องหมาย Double quote หรือ Single quote เท่านั้น 
*  sklearn เป็น module นึงของภาษา Python ที่เราจะใช้จัดการเกี่ยวกับ supervised machine learning 
*  math ที่เป็นฟังก์ชันที่ใช้คำนวณทางคณิตศาสตร์ เพื่อหาค่าทางคณิตศาสตร์ 
*  os เพื่อเช็ค dataset ว่ามีไฟล์อะไรบ้าง

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import re
import string
import pandas as pd
import sklearn as sk
import math  

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

กำหนดชุดข้อมูล train และ test ในการอ่านไฟล์และโหลดข้อมูล csv ไปยังดาต้าเฟรมและได้แสดงข้อมูลของ train เป็นจำนวน 10 แถว



In [ ]:
train = pd.read_csv('../input/nlpgettingstarted/train.csv')
test = pd.read_csv('../input/nlpgettingstarted/test.csv')
train.head(10)

และได้แสดงตารางให้เห็นจำนวนของ target 0 เป็นทวีตที่ไม่เกี่ยวข้องกับภัยพิบัติมีจำนวน 4,342 ข้อความ และ 1 เป็นทวีตที่เกี่ยวข้องกับภัยพิบัติมีจำนวน 3,271 ข้อความ

In [ ]:
temp = train.groupby('target').count()['text'].reset_index()
temp['label'] = temp['target'].apply(lambda x : 'Disaster Tweet' if x==1 else 'Non Disaster Tweet')
temp

แทนที่ช่องว่าง NaN ด้วยคำว่า 

     no_location ในคอลัมน์ location

     no_keyword ในคอลัมน์ Keyword

In [ ]:
for col in ['keyword', 'location']:
    train[col] = train[col].fillna(f'no_{col}')
for col in ['keyword', 'location']:
    test[col] = test[col].fillna(f'no_{col}')
    
    #train.head()
test.head()

# clean data

ต่อมาเราได้ทำการ clean ข้อมูล

* ขั้นแรกเราต้องการลบอิโมจิออกโดยใช้ regrex_pattern ในการตรวจสอบข้อมูลแพตเทิร์น(Pattern) คือ อิโมจิ, สัญลักษร์และภาษาภาพ, สัญลักษณ์ของการขนส่งและแผนที่ และอิโมจิรูปธงชาติใน ios และใช้ re.compile() รวบรวมรูปแบบ (Pattern) เป็นวัตถุเก็บในรูปแบบunicode แล้วทำการลบออกโดยใช้ฟังก์ชัน remove_emo
* ขั้นตอนต่อมาคือ การลบข้อความ เช่น ลบทุกตัวที่ไม่ใช่ตัวเลขและอักษรภาษาอังกฤษรวมถึงขีดล่างเทียบเท่า [A-Za-z0–9_],ขีดและอักขระ,url, แปลงอักษรตัวพิมพ์ใหญ่ทั้งหมดในสตริงข้อความให้เป็นตัวพิมพ์เล็ก,สเปซบาร์หรือแท็บหรือขึ้นบรรทัดใหม่และเครื่องหมาย " " ให้ถูกแทนที่ด้วยช่องว่าง และสุดท้ายคือ ตัวเลขที่มีเลข 0 ถึง 9
* ขั้นตอนสุดท้ายคือการลบเครื่องหมายวรรคตอนออก โดยการใช้ clean_sting ในการลบ all list ที่มีเครื่องหมายวรรคตอนอยู่ 
* และได้ apply การลบอิโมจิ ข้อความ และเครื่องหมายวรรคตอนออกทั้งในชุดฝึกฝนและชุดทดสอบ text 

และแสดงข้อมูลเป็นจำนวน 10 แถว

In [ ]:
def remove_emo(text) :
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
train['text'] = train['text'].apply(lambda x : remove_emo(x))
test['text'] = test['text'].apply(lambda x : remove_emo(x))

def removetext(text):
    text = re.sub(r'[^\w]', ' ', text)#Remove all 
    text = re.sub("\'\w+", '', text) #Remove ticks and the next character
    text = re.sub("https*\S+", " ", text)#Remove URL
    text = text.lower()
    text = re.sub('\s{2,}', " ", text)#Replace the over spaces
    text=re.sub(r'[0-9]', ' ', text)
    return text
train['text'] = train['text'].apply(lambda x : removetext(x))
test['text'] = test['text'].apply(lambda x : removetext(x))

def punctuation(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str
train['text'] = train['text'].apply(punctuation)
test['text'] = test['text'].apply(punctuation)
train.head(10)

# remove stop word
ต่อไปเราต้องการที่จะลบคำที่พบบ่อยๆ เมื่อลบแล้วแต่ยังคงความหมายเดิม ซึ่งเป็นคำที่ไม่จำเป็น เช่น of, is, the, and เป็นต้น เพื่อกรองข้อมูลที่ไร้ประโยชน์ออก 

ลบโดยการใช้คำสั่ง stop และใช้ stopword จากโมเดล nltk ที่เก็บคลังภาษาอังกฤษที่พบบ่อยๆแต่ไม่จำเป็นออกจากประโยค

In [ ]:
stop = nltk.corpus.stopwords.words('english')
train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train.head(10)

# Token
จากนั้น import sent_tokenize เพื่อตัดคำจากช่องว่าง มหัพภาค บรรทัดใหม่ และ word_tokenize เพื่อตัดคำออกมาเป็น token

และได้แสดงผลออกในตาราง text_tokens 

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
train['text_tokens'] = train['text'].apply(lambda x: word_tokenize(x))
train.head(10)

# Lemmatizer
เราได้เลือกวิธี Lemmatizer เพื่อทำให้ได้คำที่เป็นรูปแบบพื้นฐาน (รูปแบบพจนานุกรม) เพื่อให้ได้ความแม่นยำในการทำนายคำมากขึ้น

โดย import WordNetLemmatizer ใน nltk เพื่อทำ Lemmatization กับชุดฝึกฝน text_tokens และเก็บไปยังชุดฝึกฝน text_clean_tokens
 
และได้แสดงผลในตาราง text_clean_tokens

In [ ]:
from nltk.stem import WordNetLemmatizer
def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text
train['text_clean_tokens'] = train['text_tokens'].apply(lambda x: word_lemmatizer(x))
train.head(10)

จากที่เราใช้วิธี Lemmatizer เพื่อให้ได้ซึ่งคำที่ลดรูปออกมาเป็นคำที่เป็นรูปแบบพื้นฐานโดยสิ้นเชิง ที่ช่วยให้เราสามารถคำนวณได้แม่นยำ เช่น wildfires 

เมื่อเราทำวิธี stemming ก็จะได้คำว่า wildfir และเราทำวิธี Lemmatizer ก็จะได้คำว่า wildfire ซึ่งเป็นคำที่เป็นรูปแบบพื้นฐาน เป็นต้น

# CountVectorizer
เราเลือกใช้วิธีCountVwectorizer เพื่อนับจำนวนครั้งที่คำหนึ่งคำปรากฏในเอกสารซึ่งส่งผลให้เกิดการเอนเอียงในความโปรดปรานของคำที่พบบ่อยที่สุด 

สิ่งนี้กลายเป็นการละเว้นคำที่หายากซึ่งอาจช่วยได้ในการประมวลผลข้อมูลของเราอย่างมีประสิทธิภาพมากขึ้น

* ขั้นแรก import TfidfTransformer สำหรับใช้ในการคำนวณจำนวนคำ (TF) และ counterVectorizer เพื่อคำนวณ inverse document frequency (IDF) จาก sklearn.feature_extraction และทำการหา TFIDF โดยการใช้ Tfidfvectorizer ที่แปลงข้อความเป็นเวกเตอร์ 
* คำนวณค่า IDF จาก counterVectorizer แล้วเก็บใน cv ที่กำหนดการแบ่งข้อมูลเพื่อคำนวณค่าความแม่นยำหรือค่าความผิดพลาด
* แล้วทำทำกับ fit และ tranform ชุดฝึกฝน text ไปเก็บใน train_vectors และ ทำการ tranform ชุดทดสอบ text ไปยัง text_vectors



In [ ]:
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
cv = CountVectorizer()
train_vectors = cv.fit_transform(train['text'])
test_vectors = cv.transform(test["text"])

# TFIDF
เราต้องการที่จะคัดแยกคำตามความสำคัญเพื่อที่จะทำนายคำที่เกี่ยวกับภัยพิบัติจริงๆนั้น ทางกลุ่มจึงได้เลือกวิธีการ TFIDF เพื่อหาค่าการทำนาย 

โดยใช้ TfidfVectorizer และกำหนดให้           
          
          min_df = 2 หมายถึง ให้ลบคำที่พบเพียงแค่ 2 document

          max_df = 0.5 หมายถึง ให้ลบคำที่พบมากถึง 50 % ใน 1 document
และได้ทำ tfidf และทำการ fit และ tranform ค่าในชุดฝึกฝน text และเก็บใน train_vectors แล้วทำวิธีเดียวกันกับชุดทดสอบ text แล้วเก็บค่าไปยัง test_vectors 

In [ ]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 1))
train_vectors = tfidf.fit_transform(train['text'])
test_vectors = tfidf.transform(test["text"])

เราได้ import LogisticRegression และ model_selection และได้เก็บอยู่ในตัวแปร clf ที่มีเก็บฟังก์ชั่น LogisticRegression 

    และได้กำหนดตัวแปร score เพื่อคำนวณคะแนนที่ learning และทำนายออกมาใน clt โดยใช้ f1-score ซึ่งคือ 
    การหาค่าเฉลี่ยระหว่าง Precision กับ Recall และกำหนด cv = 8 ซึ่งเป็นการแบ่งข้อมูลเพื่อคำนวณค่าความแม่นยำหรือ
    ค่าความผิดพลาดจำนวน 8 รอบ เพราะรอบที่ 8 เป็นรอบที่ให้ค่า มีการทำนายที่สูงกว่ารอบอื่นๆ

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection

clf = LogisticRegression(C=3.1)
scores = model_selection.cross_val_score(clf, train_vectors, train["target"], cv=8, scoring="f1")
print(scores)

การแสดงผลความแม่นยำของ Model with Cross Validation

In [ ]:
print("Accuracy : ",scores.mean() * 100)

ปรับ simple Logistic Regression ให้เหมาะสม

In [ ]:
clf.fit(train_vectors, train["target"])

เป็นการนำผลลัพธ์ที่ได้ สุดท้ายเราทำการ predict กับ test_vectors เก็บค่าที่ได้ลงในตาราง submission ในคอลัมน์ target 

จากนั้นโหลดเข้าไฟล์ submission.csv เพื่อส่งผลการทำนาย

In [ ]:
submission = pd.read_csv('../input/nlpgettingstarted/sample_submission.csv')
submission["target"] = clf.predict(test_vectors)
submission.to_csv("submission.csv", index=False)

แสดงผลลัพธ์ของ submission จำนวน 30 แถว

In [ ]:
submission.head(30)

# Reference

arupjyoti_dutta 2020 ,Removing stop words with NLTK in Python , view 21 Nov 2020 ,https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

Katharine Jarmul 2017,Detecting Fake News with Scikit-Learn , view 21 Nov 2020 , https://www.datacamp.com/community/tutorials/scikit-learn-fake-news

koPytok 2018,Corpora/stopwords not found when import nltk library,view 21 Nov 2020,https://stackoverflow.com/questions/41610543/corpora-stopwords-not-found-when-import-nltk-library

Naim Mhedhbi 2020 , Text Classification Step by Step , view 21 Nov 2020 , https://www.kaggle.com/naim99/text-classification-step-by-step/output

Zhi Li 2019,A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model ,view 21 Nov 2020 ,https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

def love(x) 2017 ,CountVectorizer, TfidfVectorizer, Predict Comments, view 22 Nov 2020 ,https://www.kaggle.com/adamschroeder/countvectorizer-tfidfvectorizer-predict-comments#CountVectorizer----Brief-Tutorial

Jason Brownlee 2017, Deep Learning for Natural Language Processing ,view 22 Nov 2020 , https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

Kavita Ganesan 2020 , AI Implementation, Hands-On NLP Scikit-learn’s Tfidft,view 21 Nov 2020https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.X7o9lWj7SM8

# รายชื่อสมาชิก
น.ส.นันท์นภัส บุญเชิด 6209656021

น.ส.สุภาวินี แจ๊ะซ้าย 6209656039

น.ส.ณัฐนิช เพิ่มหรรษา 6209656328